# N-Arity ZipWith
###### (https://cs.brynmawr.edu/~rae/papers/2016/thesis/eisenberg-thesis.pdf)
###### (https://gist.github.com/Icelandjack/e4c86ba4d6219ad9e44a68f99319b3fa)

`Data.List` gives us functions such as `zipWith3`, `zipWith4` etc. where the user must choose the function based on the arity of the argument function. Our goal is to implement a `zipWith` function that defers this decision to the compiler.

In [28]:
:ext DataKinds

data Nat = Zero | Succ Nat

First we need a type level function that simply applies the `[]` type constructor to some type.

In [29]:
:ext TypeFamilies

type family Listify (n :: Nat) arrows where
  Listify 'Zero a = [a]
  Listify ('Succ n) (a -> b) = [a] -> Listify n b
  
[1,2,3] :: Listify 'Zero Int
_ :: Listify ('Succ 'Zero) (Int -> String)

[1,2,3]

<interactive>:1:1: error:
    • Found hole: _ :: [Int] -> [String]
    • In the expression: _ :: Listify ( 'Succ  'Zero) (Int -> String)
      In an equation for ‘it’: it = _ :: Listify ( 'Succ  'Zero) (Int -> String)
    • Relevant bindings include it :: Listify ('Succ 'Zero) (Int -> String) (bound at <interactive>:1:1)

Next we will need some runtime evidence for our choice of the number of arguments

In [38]:
:ext GADTs
:ext RankNTypes

data NumArgs :: Nat -> * -> * where
  NAZero :: forall a. NumArgs 'Zero a
  NASucc :: forall a b n. NumArgs n b -> NumArgs ('Succ n) (a -> b)

We need a GADT here so we can pattern match on the first argument to `NASucc`. This is what allows us to increment the `Nat` type parameter as well as access to the `b` parameter

We can now write the runtime workhorse

In [40]:
listApply :: NumArgs n a -> [a] -> Listify n a
listApply NAZero fs = fs
listApply (NASucc na) fs =
  \args -> listApply na (apply fs args) where
    apply :: [a -> b] -> [a] -> [b]
    apply (f:fs) (x:xs) = f x : apply fs xs
    apply _ _ = []

The `args` variable is one of the list of values to be zipped over, for instance in `[a] -> [b] -> [c]` it could be the `[a]` or `[b]`. Unless it's the last in the series, the `fs` argument will be a list functions in varies states of partial application. These functions are applied to their corresponding index in the `args` list.

To infer arity we need a function that counts the number of arrows in a function type.

In [32]:
type family CountArgs (f :: *) :: Nat where
  CountArgs (a -> b) = 'Succ (CountArgs b)
  CountArgs result = 'Zero
  
type Two = Int -> Int -> Int
_ :: Listify (CountArgs Two) Two

<interactive>:1:1: error:
    • Found hole: _ :: [Int] -> [Int] -> [Int]
    • In the expression: _ :: Listify (CountArgs Two) Two
      In an equation for ‘it’: it = _ :: Listify (CountArgs Two) Two
    • Relevant bindings include it :: Listify (CountArgs Two) Two (bound at <interactive>:1:1)

We need to connect this type level function with the term level GADT NumArgs. We use a type class to reflect type level decisions on the term level.

In [42]:
:ext MultiParamTypeClasses
:ext FlexibleInstances

class CNumArgs (numArgs :: Nat) (arrows :: *) where
  getNA :: NumArgs numArgs arrows

instance CNumArgs 'Zero a where
  getNA = NAZero
  
instance CNumArgs n b => CNumArgs ('Succ n) (a -> b) where
  getNA = NASucc getNA

We can now give the final definition of `zipWith`

In [43]:
:ext ScopedTypeVariables
:ext FlexibleContexts

zipWith :: forall f. CNumArgs (CountArgs f) f => f -> Listify (CountArgs f) f
zipWith f = listApply (getNA :: NumArgs (CountArgs f) f) (repeat f)

zipWith ((+) :: Int -> Int -> Int) [1,2,3] [2,3,4]
zipWith (\a b c -> show [a,b,c]) "abc" "123" "def"

[3,5,7]

["\"a1d\"","\"b2e\"","\"c3f\""]